In [19]:
# Importando bibliotecas
import pandas as pd 
import numpy as np 

import requests
from bs4 import BeautifulSoup as bs

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [20]:
URL = 'https://www.vivareal.com.br/venda/distrito-federal/brasilia/bairros/setor-sudoeste/apartamento_residencial/?pagina={}'
i = 1
retorno = requests.get(URL)
soup = bs(retorno.text)

###### **Verificando o tamanho de objetos carregados por página**

In [21]:
objetos_pag = soup.find_all('article', {'class':'property-card__container js-property-card'})
len(objetos_pag)

36

###### **Obtendo o primeiro objeto e verificando os atributos que serão importante e a classe ao qual pertence**

In [22]:
objeto = objetos_pag[0]
objeto

<article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="property-card__carousel js-property-carousel"> <a class="property-card__labels-container js-main-info js-listing-labels-link" href="/imovel/apartamento-4-quartos-setor-sudoeste-bairros-brasilia-com-garagem-146m2-venda-RS2250000-id-2542005292/"> <div class="property-card__inactive-listing">Indisponível</div> <div class="property-card__already-seen">Visualizado</div> </a> <div class="carousel__container js-carousel-scroll" style="width: calc(5 * 100%);"> <div class="carousel__item-wrapper js-carousel-item-wrapper"> <img alt="Foto 1 de Apartamento com 4 Quartos à venda, 146m² em Setor Sudoeste, Brasília" class="carousel__image js-carousel-image" src="https://resizedimgs.vivareal.com/crop/286x200/vr.images.sp/b53ff5454ff66192c2aeb0a879fa382c.jpg" title="Apartamento com 4 Quartos à venda, 146m² em Setor Sudoeste, Brasília - i

###### **Colocando cada objeto dentro de uma variável**

In [23]:
qtd_imoveis = float(soup.find('strong', {'class':'results-summary__count js-total-records'}).text.replace('.',''))
descricao = soup.find('span', {'class': 'property-card__title js-cardLink js-card-title'}).text.strip()
endereco = soup.find('span', {'class': 'property-card__address'}).text.strip()
metragem = int(soup.find('li', {'class': 'property-card__detail-item property-card__detail-area'}).span.text.strip())
quartos = int(soup.find('li', {'class': 'property-card__detail-room'}).span.text.strip())
banheiro = int(soup.find('li',{'class': 'property-card__detail-bathroom'}).span.text.strip())
vagas = int(soup.find('li',{'class': 'property-card__detail-garage'}).span.text.strip())
valor = soup.find(class_='property-card__price js-property-card-prices js-property-card__price-small').text.strip()
condominio = soup.find('strong', {'class':'js-condo-price'}).text.strip()

print(qtd_imoveis)
print(descricao)
print(endereco)
print(metragem)
print(quartos)
print(banheiro)
print(vagas)
print(valor)
print(condominio)

94.0
Apartamento com 4 Quartos à Venda, 146m²
SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasília - DF
146
4
5
2
R$ 2.250.000
R$ 1.200


###### **Criando Dataframe para organizar as variáveis encontradas. Deixei a quantidade de imóveis encontrados na pesquisa(64) fora do Dataframe**

In [24]:
df = pd.DataFrame(columns=[
    'descricao',
    'endereco',
    'metragem',
    'quartos',
    'banheiro',
    'vagas',
    'valor',
    'condominio',
])

i = 0

In [25]:
df.columns

Index(['descricao', 'endereco', 'metragem', 'quartos', 'banheiro', 'vagas',
       'valor', 'condominio'],
      dtype='object')

###### **Laço de repetição para inserir os 64 objetos encontrados**

In [26]:

while qtd_imoveis > df.shape[0]:
    
    i += 1
    print(f"valor i: {i} \t\t qtd_imoveis: {df.shape[0]}")
    ret = requests.get(URL.format(i))
    soup = bs(ret.text)

    final = soup.find_all('article', {'class':'property-card__container js-property-card'})

    for objeto in final:
        try:
            descricao = objeto.find('span', {'class': 'property-card__title js-cardLink js-card-title'}).text.strip()
        except: 
            descricao = None
        try:
            endereco = objeto.find('span', {'class': 'property-card__address'}).text.strip()
        except: 
            endereco = None
        try:
            metragem = int(objeto.find('li', {'class': 'property-card__detail-item property-card__detail-area'}).span.text.strip())
        except:
            metragem = None
        try:
            quartos = int(objeto.find('li', {'class': 'property-card__detail-room'}).span.text.strip())
        except: 
            quartos = None
        try: 
            banheiro = int(objeto.find('li',{'class': 'property-card__detail-bathroom'}).span.text.strip())
        except:
            banheiro = None
        try:
            vagas = int(objeto.find('li',{'class': 'property-card__detail-garage'}).span.text.strip())
        except:
            vagas = None
        try:
            valor = objeto.find(class_='property-card__price js-property-card-prices js-property-card__price-small').text.strip()
        except:
            valor = None
        try:
            condominio = objeto.find('strong', {'class':'js-condo-price'}).text.strip()
        except:
            condominio = None
            
        df.loc[df.shape[0]] = [
            descricao,
            endereco,
            metragem,
            quartos,
            banheiro,
            vagas,
            valor,
            condominio
        ]      

valor i: 1 		 qtd_imoveis: 0
valor i: 2 		 qtd_imoveis: 36
valor i: 3 		 qtd_imoveis: 72


In [27]:
df.head(20)

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio
0,"Apartamento com 4 Quartos à Venda, 146m²","SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasíl...",146,4,5,2,R$ 2.250.000,R$ 1.200
1,Haus by Novka,"Parque Recreativo Sarah Kubitschek, - - Asa Su...",None,None,None,1,Sob Consulta,None
2,"Apartamento com 3 Quartos à Venda, 100m²","Setor Sudoeste, Brasília - DF",100,3,3,1,R$ 1.350.000,None
3,"Apartamento com 3 Quartos à Venda, 86m²","SQSW 105 Bloco F - Setor Sudoeste, Brasília - DF",86,3,3,1,R$ 1.150.000,R$ 860
4,"Apartamento com Quarto à Venda, 24m²","QMSW 5, 5 - Setor Sudoeste, Brasília - DF",24,1,1,1,R$ 280.000,None
5,"Apartamento com 2 Quartos à Venda, 90m²","SQSW 101 Bloco H, 101 - Setor Sudoeste, Brasíl...",90,2,2,1,R$ 1.300.000,None
6,"Apartamento com 3 Quartos à Venda, 90m²","SQSW 100 Bloco D, 1 - Setor Sudoeste, Brasília...",90,3,3,1,R$ 1.255.000,R$ 820
7,"Apartamento com 3 Quartos à Venda, 102m²","SQSW 304 Bloco D, 304 - Setor Sudoeste, Brasíl...",102,3,3,1,R$ 1.200.000,R$ 711
8,Parque das Figueiras,"SQNW 103 Bloco G, SQNW 103 - Setor Noroeste, B...",102,3,4,2,R$ 1.558.560,None
9,"Apartamento com 3 Quartos à Venda, 89m²","SQSW 100 Bloco F, 603 - Setor Sudoeste, Brasíl...",89,3,1,1,R$ 1.100.000,R$ 800.000


##### Verificando as linhas nulas

In [28]:
df[df.isnull().any(axis=1)]

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio
1,Haus by Novka,"Parque Recreativo Sarah Kubitschek, - - Asa Su...",None,None,None,1,Sob Consulta,None
2,"Apartamento com 3 Quartos à Venda, 100m²","Setor Sudoeste, Brasília - DF",100,3,3,1,R$ 1.350.000,None
4,"Apartamento com Quarto à Venda, 24m²","QMSW 5, 5 - Setor Sudoeste, Brasília - DF",24,1,1,1,R$ 280.000,None
5,"Apartamento com 2 Quartos à Venda, 90m²","SQSW 101 Bloco H, 101 - Setor Sudoeste, Brasíl...",90,2,2,1,R$ 1.300.000,None
8,Parque das Figueiras,"SQNW 103 Bloco G, SQNW 103 - Setor Noroeste, B...",102,3,4,2,R$ 1.558.560,None
10,"Apartamento com 3 Quartos à Venda, 146m²","Setor Sudoeste, Brasília - DF",146,3,4,2,R$ 1.820.000,None
13,"Apartamento com 4 Quartos à Venda, 144m²","SQSW 500, 5 - Setor Sudoeste, Brasília - DF",144,4,5,3,R$ 2.115.314,None
15,Parque dos Cedros,"SQNW 306 Bloco A, s/n - Setor Noroeste, Brasíl...",165,4,6,4,R$ 2.681.463,None
19,"Apartamento com Quarto à Venda, 36m²","QRSW 4, 4 - Setor Sudoeste, Brasília - DF",36,1,1,None,R$ 288.000,R$ 380
21,"Apartamento com 4 Quartos à Venda, 244m²","SQSW 500, 7 - Setor Sudoeste, Brasília - DF",244,4,6,4,R$ 3.454.653,None


##### Filtrando as linhas nulas

In [29]:
filtro = df[df.isnull().any(axis=1)].index
filtro

Int64Index([  1,   2,   4,   5,   8,  10,  13,  15,  19,  21,  25,  28,  29,
             31,  32,  37,  38,  40,  41,  44,  46,  49,  51,  55,  57,  61,
             64,  65,  67,  68,  73,  74,  76,  77,  80,  82,  85,  87,  91,
             93,  97, 100, 101, 103, 104],
           dtype='int64')

##### Apagando as linhas nulas

In [30]:
df_new = df.drop(filtro)
df_new.head(20)

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio
0,"Apartamento com 4 Quartos à Venda, 146m²","SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasíl...",146,4,5,2,R$ 2.250.000,R$ 1.200
3,"Apartamento com 3 Quartos à Venda, 86m²","SQSW 105 Bloco F - Setor Sudoeste, Brasília - DF",86,3,3,1,R$ 1.150.000,R$ 860
6,"Apartamento com 3 Quartos à Venda, 90m²","SQSW 100 Bloco D, 1 - Setor Sudoeste, Brasília...",90,3,3,1,R$ 1.255.000,R$ 820
7,"Apartamento com 3 Quartos à Venda, 102m²","SQSW 304 Bloco D, 304 - Setor Sudoeste, Brasíl...",102,3,3,1,R$ 1.200.000,R$ 711
9,"Apartamento com 3 Quartos à Venda, 89m²","SQSW 100 Bloco F, 603 - Setor Sudoeste, Brasíl...",89,3,1,1,R$ 1.100.000,R$ 800.000
11,"Apartamento com 2 Quartos à Venda, 68m²","CCSW 3, 3 - Setor Sudoeste, Brasília - DF",68,2,2,1,R$ 935.000,R$ 733
12,"Apartamento com 3 Quartos à Venda, 87m²","SQSW 301 Bloco E, 1 - Setor Sudoeste, Brasília...",87,3,3,1,R$ 1.190.000,R$ 980
14,"Apartamento com Quarto à Venda, 26m²","CCSW 1, 4 - Setor Sudoeste, Brasília - DF",26,1,1,1,R$ 510.000,R$ 410
16,"Apartamento com 3 Quartos à Venda, 95m²","SQSW 305 Bloco F, 01 - Setor Sudoeste, Brasíli...",95,3,3,1,R$ 1.220.000,R$ 923
17,"Apartamento com 2 Quartos à Venda, 65m²","SQSW 104 Bloco G, 104 - Setor Sudoeste, Brasíl...",65,2,2,1,R$ 1.049.000,R$ 530


In [31]:
df_new.query("valor == 'Sob Consulta'")

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio


###### **Limpeza dos dados.**

In [32]:

df_new['valor'] = df_new['valor'].apply(lambda x: x.replace('R$','').strip())
df_new['condominio'] = df_new['condominio'].replace(np.nan, 'R$ 0.0')
df_new['condominio'] = df_new['condominio'].apply(lambda x: x.replace('R$','').strip())



df_new.head(10)

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio
0,"Apartamento com 4 Quartos à Venda, 146m²","SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasíl...",146,4,5,2,2.250.000,1.200
3,"Apartamento com 3 Quartos à Venda, 86m²","SQSW 105 Bloco F - Setor Sudoeste, Brasília - DF",86,3,3,1,1.150.000,860
6,"Apartamento com 3 Quartos à Venda, 90m²","SQSW 100 Bloco D, 1 - Setor Sudoeste, Brasília...",90,3,3,1,1.255.000,820
7,"Apartamento com 3 Quartos à Venda, 102m²","SQSW 304 Bloco D, 304 - Setor Sudoeste, Brasíl...",102,3,3,1,1.200.000,711
9,"Apartamento com 3 Quartos à Venda, 89m²","SQSW 100 Bloco F, 603 - Setor Sudoeste, Brasíl...",89,3,1,1,1.100.000,800.000
11,"Apartamento com 2 Quartos à Venda, 68m²","CCSW 3, 3 - Setor Sudoeste, Brasília - DF",68,2,2,1,935.000,733
12,"Apartamento com 3 Quartos à Venda, 87m²","SQSW 301 Bloco E, 1 - Setor Sudoeste, Brasília...",87,3,3,1,1.190.000,980
14,"Apartamento com Quarto à Venda, 26m²","CCSW 1, 4 - Setor Sudoeste, Brasília - DF",26,1,1,1,510.000,410
16,"Apartamento com 3 Quartos à Venda, 95m²","SQSW 305 Bloco F, 01 - Setor Sudoeste, Brasíli...",95,3,3,1,1.220.000,923
17,"Apartamento com 2 Quartos à Venda, 65m²","SQSW 104 Bloco G, 104 - Setor Sudoeste, Brasíl...",65,2,2,1,1.049.000,530


In [33]:
df_new.to_csv('imoveis.csv', sep=',', index=False)